### Virtual Machine

In [ ]:
# Activate Cloud Shell
gcloud auth list

gcloud config list project

# Set the region and zone
gcloud config set compute/region "region=$REGION"

export REGION=region=$REGION

export ZONE=Zone

# NGINX web server
sudo apt-get update

sudo apt-get install -y nginx

ps auwx | grep nginx

# Create a new instance
gcloud compute instances create gcelab2 --machine-type e2-medium \
    --zone=$ZONE

gcloud compute instances create --help

gcloud compute ssh gcelab2 --zone=$ZONE

### Cloud Shell

In [ ]:
# Activate Cloud Shell
gcloud auth list

gcloud config list project

# Configure regions and zones
gcloud config set compute/region us-east1

gcloud config get-value compute/region

gcloud config set compute/zone us-east1-c

gcloud config get-value compute/zone

# Project information
gcloud config get-value project

gcloud compute project-info describe --project \
    $(gcloud config get-value project)

# Environment variables
export PROJECT_ID=$(gcloud config get-value project)

export ZONE=$(gcloud config get-value compute/zone)

echo -e "PROJECT ID: $PROJECT_ID\nZONE: $ZONE"

# Creating a virtual machine with the gcloud tool
gcloud compute instances create gcelab2 --machine-type e2-medium \
    --zone $ZONE

gcloud compute instances create --help

gcloud -h

gcloud config --help

gcloud help config

gcloud config list

gcloud config list --all

gcloud components list

### Kubernetes Engine

In [ ]:
# Activate Cloud Shell
gcloud auth list

gcloud config list project

# Set a default compute zone
gcloud config set compute/region us-east4

gcloud config set compute/zone us-east4-c

# Create GKE cluster
gcloud container clusters create --machine-type=e2-medium \
    --zone=us-east4-c lab-cluster

# Get authentication credentials for the cluster
gcloud container clusters get-credentials lab-cluster 

# Deploy an application to cluster
kubectl create deployment hello-server \
    --image=gcr.io/google-samples/hello-app:1.0
    
kubectl expose deployment hello-server \
    --type=LoadBalancer --port 8080

kubectl get service

# Delete the cluster
gcloud container clusters delete lab-cluster

### Network & HTTP Load Balancers

In [ ]:
# Activate Cloud Shell
gcloud auth list

gcloud config list project

# Set default region and zone
gcloud config set compute/region us-east4

gcloud config set compute/zone us-east4-b

# Create multiple web server instances
gcloud compute instances create www1 \
    --zone=us-east4-b \
    --tags=network-lb-tag \
    --machine-type=e2-small \
    --image-family=debian-11 \
    --image-project=debian-cloud \
    --metadata=startup-script='#!/bin/bash
        apt-get update
        apt-get install apache2 -y
        service apache2 restart
        echo "<h3>Web Server: www1</h3>" \
            | tee /var/www/html/index.html
        '
# Create firewall rule
gcloud compute firewall-rules create www-firewall-network-lb \
    --target-tags network-lb-tag --allow tcp:80

# Get external IP address 
gcloud compute instances list

curl http://[IP_ADDRESS]
    
# Configure load balancing service
# Create a static external IP address
gcloud compute addresses create network-lb-ip-1 \
    --region us-east4

# Add a legacy HTTP health check
gcloud compute http-health-checks create basic-check

# Create target pool
gcloud compute target-pools create www-pool \
    --region us-east4 --http-health-check basic-check

# Add the instances to the pool
gcloud compute target-pools add-instances www-pool \
    --instances www1,www2,www3

# Add a forwarding rule
gcloud compute forwarding-rules create www-rule \
    --region us-east4 \
    --ports 80 \
    --address network-lb-ip-1 \
    --target-pool www-pool

# Sending traffic to instances
# View the external IP address of forwarding rule
gcloud compute forwarding-rules describe www-rule --region us-east4

IPADDRESS=$(gcloud compute forwarding-rules describe www-rule 
    --region us-east4 --format="json" | jq -r .IPAddress)

echo $IPADDRESS

while true; do curl -m1 $IPADDRESS; done

# Create HTTP Load Balancer
# Create template
gcloud compute instance-templates create lb-backend-template \
    --region=us-east4 \
    --network=default \
    --subnet=default \
    --tags=allow-health-check \
    --machine-type=e2-medium \
    --image-family=debian-11 \
    --image-project=debian-cloud \
    --metadata=startup-script='#!/bin/bash
        apt-get update
        apt-get install apache2 -y
        a2ensite default-ssl
        a2enmod ssl
        vm_hostname="$(curl -H "Metadata-Flavor:Google" \
            http://169.254.169.254/computeMetadata/v1/instance/name)"
        echo "Page served from: $vm_hostname" | \
            tee /var/www/html/index.html
        systemctl restart apache2
    '

# Create managed instance groups
gcloud compute instance-groups managed create lb-backend-group \
    --template=lb-backend-template --size=2 --zone=us-east4-b

# Create firewall rule
gcloud compute firewall-rules create fw-allow-health-check \
    --network=default \
    --action=allow \
    --direction=ingress \
    --source-ranges=130.211.0.0/22,35.191.0.0/16 \
    --target-tags=allow-health-check \
    --rules=tcp:80

# Set up global static external IP address
gcloud compute addresses create lb-ipv4-1 \
    --ip-version=IPV4 \
    --global

gcloud compute addresses describe lb-ipv4-1 \
    --format="get(address)" \
    --global

# Create health check
gcloud compute health-checks create http http-basic-check \
    --port 80

# Create backend service
gcloud compute backend-services create web-backend-service \
    --protocol=HTTP \
    --port-name=http \
    --health-checks=http-basic-check \
    --global

# Add instance group to the backend service
gcloud compute backend-services add-backend web-backend-service \
    --instance-group=lb-backend-group \
    --instance-group-zone=us-east4-b \
    --global

# Create URL map to route the incoming requests
gcloud compute url-maps create web-map-http \
    --default-service web-backend-service

# Create a target HTTP proxy
gcloud compute target-http-proxies create http-lb-proxy \
    --url-map web-map-http

# Create a global forwarding rule to route \
# incoming requests to the proxy
gcloud compute forwarding-rules create http-content-rule \
    --address=lb-ipv4-1 \
    --global \
    --target-http-proxy=http-lb-proxy \
    --ports=80

### Challenge Lab

In [ ]:
export INSTANCE_NAME=nucleus-jumphost-278
export ZONE=us-west3-c
export REGION=us-west3
export PORT=8082
export FIREWALL_NAME=accept-tcp-rule-747

gcloud config set compute/region $REGION
gcloud config set compute/zone $ZONE

# Task 1. Create a new instance
gcloud compute instances create $INSTANCE_NAME \
    --machine-type=f1-micro

# Task 2. Create GKE cluster
gcloud container clusters create nucleus-cluster \
    --machine-type=n1-standard-1

# Get authentication credentials for the cluster
gcloud container clusters get-credentials nucleus-cluster

# Deploy application to cluster
kubectl create deployment hello-app \
    --image=gcr.io/google-samples/hello-app:2.0

# Expose the app
kubectl expose deployment hello-app \
    --type=LoadBalancer \
    --port=$PORT

# Task 3. Set up an HTTP load balancer

cat << EOF > startup.sh
#! /bin/bash
apt-get update
apt-get install -y nginx
service nginx start
sed -i -- 's/nginx/Google Cloud Platform - '"\$HOSTNAME"'/' \
    /var/www/html/index.nginx-debian.html
EOF

# Create an instance template
gcloud compute instance-templates create nucleus-lb-template \
    --network=default \
    --machine-type=n1-standard-1 \
    --metadata-from-file=startup-script=startup.sh

# Create a target pool
gcloud compute target-pools create nucleus-lb-pool

# Create a managed instance group
gcloud compute instance-groups managed create nucleus-lb-group \
    --template=nucleus-lb-template \
    --size=2

# Create a firewall rule
gcloud compute firewall-rules create $FIREWALL_NAME \
    --network=default \
    --action=allow \
    --rules=tcp:80
    
# Create a health check
gcloud compute health-checks create http http-basic-check \
    --port=80

# Create a backend service
gcloud compute backend-services create web-backend-service \
    --protocol=HTTP \
    --port-name=http \
    --health-checks=http-basic-check \
    --global

# Attach the managed instance group
gcloud compute backend-services add-backend web-backend-service \
    --instance-group=nucleus-lb-group \
    --instance-group-zone=$ZONE \
    --global

# Create a URL map & route requests
gcloud compute url-maps create web-map-http \
    --default-service=web-backend-service

# Route requests to URL map
gcloud compute target-http-proxies create http-lb-proxy \
    --url-map=web-map-http

# Create a forwarding rule
gcloud compute forwarding-rules create http-content-rule \
    --target-http-proxy=http-lb-proxy \
    --ports=80 \
    --global